<a href="https://colab.research.google.com/github/clove10002/SimpleAssets/blob/master/uploadvideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r requirements.txt

import os
import io
import requests
import google.auth
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload

# Define API scopes
SCOPES = ['https://www.googleapis.com/auth/drive.readonly', 'https://www.googleapis.com/auth/youtube.upload']
CLIENT_SECRET_FILE = 'credentials.json'
TOKEN_FILE = 'token.json'

# URL to fetch npoint.io data
NPOINT_URL = "https://api.npoint.io/9f73dfc9e60ba129495e"

# Authenticate Google API
def authenticate_google():
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds, _ = google.auth.load_credentials_from_file(TOKEN_FILE)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())
    return creds

# Extract file ID from Google Drive URL
def extract_file_id(url):
    try:
        return url.split('/d/')[1].split('/')[0]
    except IndexError:
        print("Invalid Google Drive URL format.")
        return None

# Function to get file stream from Google Drive
def get_drive_file_stream(file_id, service):
    request = service.files().get_media(fileId=file_id)
    file_stream = io.BytesIO()
    downloader = MediaIoBaseDownload(file_stream, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        print(f"Streaming {int(status.progress() * 100)}%.")

    file_stream.seek(0)  # Reset stream position
    return file_stream  # Return the file as a stream

# Upload video stream to YouTube
def upload_stream_to_youtube(file_stream, title, description, tags, youtube):
    print(f"Uploading {title} to YouTube...")

    body = {
        'snippet': {
            'title': title,
            'description': description,
            'tags': tags,
        },
        'status': {
            'privacyStatus': 'public',
        }
    }

    media = MediaIoBaseUpload(file_stream, mimetype='video/*', resumable=True)
    request = youtube.videos().insert(part="snippet,status", body=body, media_body=media)

    response = request.execute()
    print(f"Upload successful! Video ID: {response['id']}")

# Fetch data from npoint.io
def fetch_npoint_data():
    response = requests.get(NPOINT_URL)
    if response.status_code == 200:
        return response.json().get("data", [])
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return []

# Main function
def main():
    creds = authenticate_google()
    drive_service = build('drive', 'v3', credentials=creds)
    youtube_service = build('youtube', 'v3', credentials=creds)

    videos = fetch_npoint_data()
    if not videos:
        print("No videos to process.")
        return

    for video in videos:
        file_id = extract_file_id(video['url'])
        if file_id:
            file_stream = get_drive_file_stream(file_id, drive_service)  # Get video stream
            upload_stream_to_youtube(file_stream, video['title'], video['description'],
                                     [tag['tag'] for tag in video['tag']], youtube_service)

if __name__ == '__main__':
    main()